In [ ]:
import json
import os
import re
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [2]:
files = os.listdir('data')

In [3]:
stations_files = [file_name for file_name in files if 'stations' in file_name]
status_files = [file_name for file_name in files if 'status' in file_name]

In [ ]:
stations_data_list = []

for file in tqdm(stations_files, desc='file'):
    
    with open(f'data/{file}', 'r') as file_object:
        try:
            stations_json_load = json.load(file_object)
        except json.JSONDecodeError as e:
            print(f'error on {file}: {e}')
            continue

    time_stations_data = (
        pd.json_normalize(
            data=stations_json_load,
            record_path= [
                ['data','stations']
            ],
            meta='last_updated',
        )
        .filter(items=[
            'last_updated',
            'station_id',
            'legacy_id',
            'name',
            'capacity',
            'lat',
            'lon'
        ])
    )

    file_timestamp = int(re.search('\d*',file).group())

    time_stations_data['status_last_updated_fetched_timestamp'] = file_timestamp 


    stations_data_list.append(time_stations_data)

stations_data = pd.concat(stations_data_list)

In [ ]:
status_data_list = []

for file in tqdm(status_files):

    with open(f'data/{file}', 'r') as file_object:
        try:
            status_json_load = json.load(file_object)
        except json.JSONDecodeError as e:
            print(f'error on {file}: {e}')
            continue

    time_status_data = pd.json_normalize(
        data=status_json_load,
        record_path= [
            ['data','stations']
        ],
        meta='last_updated',
    ).filter(items=[
        'last_updated',
        'station_id',
        'station_status',
        'is_renting',
        'is_returning',
        'num_docks_available',
        'num_bikes_available',
        'num_ebikes_available',
        'num_bikes_disabled',
        'num_docks_disabled',
        'valet.active'
    ])

    status_data_list.append(time_status_data)


status_data = pd.concat(status_data_list)


localize times

In [126]:
status_data['last_updated'] = (
    status_data['last_updated']
    .apply(pd.Timestamp, unit='s', tz='US/Eastern')
)

stations_data['status_last_updated_fetched_timestamp'] = (
    stations_data['status_last_updated_fetched_timestamp']
    .apply(pd.Timestamp, unit='s', tz='US/Eastern')
)

check that stations data timestamps have matching status data timestamps

In [130]:
assert stations_data['status_last_updated_fetched_timestamp'].isin(status_data['last_updated']).all()

In [131]:
dataset = (
    stations_data
    .merge(
        status_data, 
        left_on=['status_last_updated_fetched_timestamp','station_id'],
        right_on=['last_updated','station_id'],
        how='inner',
        suffixes=['_stations',None]
    )
)

check that each station_id is a unique physical location

In [132]:
assert dataset.groupby('station_id')['lat'].nunique().max() == 1

In [133]:
stations_locations = (
    dataset
    .drop_duplicates(subset='station_id')
    .set_index('station_id')
    [['lat','lon']]
)

In [134]:
stations_geo = gpd.GeoDataFrame(
    index=stations_locations.index,
    geometry=gpd.points_from_xy(
        stations_locations['lon'],
        stations_locations['lat'],
        crs='epsg:4326'
    )
)

In [136]:
dataset = (
    dataset
    .set_index(['last_updated','station_id'])
    .drop(columns=[
        'name',
        'lat','lon',
        'status_last_updated_fetched_timestamp',
        'last_updated_stations',
        ])
)

In [137]:
dataset.head(10)

capacity  \
last_updated              station_id                                       
2024-03-12 17:40:49-04:00 66dc2995-0aca-11e7-82f6-3863bb44ef7c        51   
                          06439006-11b6-44f0-8545-c9d39035f32a        48   
                          19d17911-1e4a-41fa-b62b-719aa0a6182e        39   
                          1861678548643203686                         25   
                          cd2d9dab-7708-4685-a56f-9412c738de7e        23   
                          66db2a71-0aca-11e7-82f6-3863bb44ef7c        45   
                          901ad0c4-383e-490a-8b54-0656ce2358d6        19   
                          66dc292c-0aca-11e7-82f6-3863bb44ef7c        55   
                          4c03fa2d-89da-4f0b-8f19-c7de5edbe256        25   
                          9af90faf-0b9b-451b-9cb0-20ff421ca1d9        27   

                                                                is_renting  \
last_updated              station_id                                         
2024-03-12 17:40:49-04:00 66dc2995-0aca-11e7-82f6-3863bb44ef7c           0   
                          06439006-11b6-44f0-8545-c9d39035f32a           0   
                          19d17911-1e4a-41fa-b62b-719aa0a6182e           0   
                          1861678548643203686                            1   
                          cd2d9dab-7708-4685-a56f-9412c738de7e           1   
                          66db2a71-0aca-11e7-82f6-3863bb44ef7c           1   
                          901ad0c4-383e-490a-8b54-0656ce2358d6           1   
                          66dc292c-0aca-11e7-82f6-3863bb44ef7c           1   
                          4c03fa2d-89da-4f0b-8f19-c7de5edbe256           1   
                          9af90faf-0b9b-451b-9cb0-20ff421ca1d9           1   

                                                                is_returning  \
last_updated              station_id                                           
2024-03-12 17:40:49-04:00 66dc2995-0aca-11e7-82f6-3863bb44ef7c             0   
                          06439006-11b6-44f0-8545-c9d39035f32a             0   
                          19d17911-1e4a-41fa-b62b-719aa0a6182e             0   
                          1861678548643203686                              1   
                          cd2d9dab-7708-4685-a56f-9412c738de7e             1   
                          66db2a71-0aca-11e7-82f6-3863bb44ef7c             1   
                          901ad0c4-383e-490a-8b54-0656ce2358d6             1   
                          66dc292c-0aca-11e7-82f6-3863bb44ef7c             1   
                          4c03fa2d-89da-4f0b-8f19-c7de5edbe256             1   
                          9af90faf-0b9b-451b-9cb0-20ff421ca1d9             1   

                                                                num_docks_available  \
last_updated              station_id                                                  
2024-03-12 17:40:49-04:00 66dc2995-0aca-11e7-82f6-3863bb44ef7c                    0   
                          06439006-11b6-44f0-8545-c9d39035f32a                    0   
                          19d17911-1e4a-41fa-b62b-719aa0a6182e                    0   
                          1861678548643203686                                     8   
                          cd2d9dab-7708-4685-a56f-9412c738de7e                    1   
                          66db2a71-0aca-11e7-82f6-3863bb44ef7c                    0   
                          901ad0c4-383e-490a-8b54-0656ce2358d6                    3   
                          66dc292c-0aca-11e7-82f6-3863bb44ef7c                   50   
                          4c03fa2d-89da-4f0b-8f19-c7de5edbe256                    3   
                          9af90faf-0b9b-451b-9cb0-20ff421ca1d9                   18   

                                                                num_bikes_available  \
last_updated              station_id                                                  
2024-03-12 17:40:49-04:00 66dc2995

save out

In [ ]:
dataset.to_parquet('dataset.parquet') 

In [ ]:
stations_geo.to_file('stations_geo.geojson')